In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# 데이터 로드
train = pd.read_csv('/kaggle/input/2021-ai-quiz1-p2/train.csv')
test = pd.read_csv('/kaggle/input/2021-ai-quiz1-p2/test.csv')
submit = pd.read_csv('/kaggle/input/2021-ai-quiz1-p2/submit_sample.csv')

# 불필요한 열 제거 및 라벨 분리
label = train['8']
train = train.drop(['Unnamed: 0','8'],axis=1)
test = test.drop(['Unnamed: 0','8'],axis=1)
print(train.shape, test.shape, label.shape)

# standard scaler 를 이용한 전처리
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_std = scaler.fit_transform(train)
test_std = scaler.transform(test)
# 랜덤 시드 고정 및 gpu 설정
import torch
import random
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)
random.seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 텐서로 변환
X = torch.FloatTensor(train_std).to(device)
Y = torch.FloatTensor(label).to(device)
print(X.shape, Y.shape)

# 모델 설정 및 활성화 함수 설정
linear1 = nn.Linear(8,1,bias=True)
sigmoid = nn.Sigmoid()

model = nn.Sequential(linear1, sigmoid).to(device)
# 옵티마이저 및 손실함수 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.BCELoss().to(device)
# 모델 학습
model.train()
epochs = 1000

for epoch in range(epochs + 1):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_fn(output, Y.unsqueeze(1))
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch: {:4d}, Cost: {:6f}".format(epoch, loss.item()))
Epoch:    0, Cost: 0.715863
Epoch:  100, Cost: 0.608985
Epoch:  200, Cost: 0.558321
Epoch:  300, Cost: 0.530702
Epoch:  400, Cost: 0.513910
Epoch:  500, Cost: 0.502915
Epoch:  600, Cost: 0.495343
Epoch:  700, Cost: 0.489937
Epoch:  800, Cost: 0.485973
Epoch:  900, Cost: 0.483007
Epoch: 1000, Cost: 0.480751
# 텐서로 변환
X_test = torch.FloatTensor(test_std).to(device)
# 모델 평가
model.eval()
with torch.no_grad():
    predict = model(X_test)
# 이진 분류를 위해 0,1 값으로 변환
pred = (predict.cpu() >= torch.FloatTensor([0.5])).to(int)
# 제출
submit['Label'] = pred.detach().numpy()
submit.to_csv("submit-2.csv",index=False)
분석 결과
standard scaler를 이용해 데이터를 전처리하였을 때 더 높은 성능을 볼 수 있었다. 또한 learning rate가 너무 작으면 학습이 잘 안되었다. epoch를 크게 키워도 큰 성능 차이는 없었고 비슷한 cost 수준에서 계속 수렴하는 듯 했다. 오히려 learning rate를 조작하였을 때 성능 차이를 볼 수 있었다. 그리고 레이어를 쌓을 수록 성능이 안좋아졌다. 한개의 레이어만 있을 때 가장 높은 성능이 나왔고 그렇지 않을 때에는 오히려 과적합 문제가 있었던것 같았다. 그리고 첫번째 아이디 열을 제거했을 때 더 높은 성능이 나왔다. 불필요한 데이터를 제거해야한다.

 
 